In [20]:
import json
import os
import uuid
from typing import List, Dict, Tuple, Optional, Any
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue, Range
import rank_bm25
from rank_bm25 import BM25Okapi
from mistralai import Mistral

import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

In [36]:
# Конфигурация
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
QDRANT_COLLECTION_EMBEDDINGS = "telegram_embeddings"
QDRANT_COLLECTION_BM25 = "telegram_bm25"
VECTOR_SIZE = 384

class TelegramRAGSystem:
    def __init__(self, data_dir: str = "./rag_data", mistral_api_key: str = None):
        self.data_dir = data_dir
        self.embedding_model = SentenceTransformer(EMBEDDING_MODEL)
        
        # Инициализация Qdrant
        self.qdrant_embeddings = QdrantClient(path=os.path.join(data_dir, "embeddings_db"))
        self.qdrant_bm25 = QdrantClient(path=os.path.join(data_dir, "bm25_db"))
        
        # BM25
        self.bm25 = None
        self.bm25_documents = []
        self.bm25_doc_ids = []  # Храним оригинальные ID
        self.bm25_uuid_map = {}  # Маппинг UUID -> оригинальный ID
        
        # Mistral клиент
        self.mistral_client = Mistral(api_key=mistral_api_key) if mistral_api_key else None
        
        self._initialize_collections()
        self._load_bm25_data()
    
    def _generate_uuid(self, original_id: str) -> str:
        """Генерация UUID на основе оригинального ID"""
        return str(uuid.uuid5(uuid.NAMESPACE_DNS, original_id))
    
    def _initialize_collections(self):
        """Инициализация коллекций в Qdrant"""
        # Коллекция для эмбеддингов
        if not self.qdrant_embeddings.collection_exists(QDRANT_COLLECTION_EMBEDDINGS):
            self.qdrant_embeddings.create_collection(
                collection_name=QDRANT_COLLECTION_EMBEDDINGS,
                vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
            )
        
        # Коллекция для BM25
        if not self.qdrant_bm25.collection_exists(QDRANT_COLLECTION_BM25):
            self.qdrant_bm25.create_collection(
                collection_name=QDRANT_COLLECTION_BM25,
                vectors_config=VectorParams(size=1, distance=Distance.COSINE)
            )
    
    def _load_bm25_data(self):
        """Загрузка данных для BM25 из Qdrant"""
        points = self.qdrant_bm25.scroll(
            collection_name=QDRANT_COLLECTION_BM25,
            limit=10000
        )[0]
        
        documents = []
        doc_ids = []
        
        for point in points:
            if point.payload and 'text' in point.payload and 'original_id' in point.payload:
                documents.append(self._tokenize_text(point.payload['text']))
                doc_ids.append(point.payload['original_id'])
                self.bm25_uuid_map[point.id] = point.payload['original_id']
        
        if documents:
            self.bm25 = BM25Okapi(documents)
            self.bm25_documents = documents
            self.bm25_doc_ids = doc_ids
    
    def _tokenize_text(self, text: str) -> List[str]:
        """
        Продвинутая токенизация текста для BM25
        Args:
            text: входной текст для токенизации
        Returns:
            List[str]: список токенов (основ слов)
        """
        self.stemmer = SnowballStemmer('russian')
        # Паттерн для выделения слов (буквы, цифры, дефисы, апострофы)
        self.word_pattern = re.compile(r"[a-zA-Zа-яА-ЯёЁ0-9]+(?:[-'’][a-zA-Zа-яА-ЯёЁ0-9]+)*")
        
        # Приводим к нижнему регистру
        text = text.lower()
        # Извлекаем слова с помощью регулярного выражения
        words = self.word_pattern.findall(text)
        # Применяем стемминг к каждому слову
        tokens = [self.stemmer.stem(word) for word in words]
        # Фильтруем слишком короткие токены (опционально)
        tokens = [token for token in tokens if len(token) > 2]
        
        return tokens
    
    def add_documents(self, documents: List[Dict[str, Any]]):
        """
        Добавление новых документов в обе базы данных
        """
        points_embeddings = []
        points_bm25 = []
        new_bm25_docs = []
        
        for doc in documents:
            # Генерация UUID
            doc_uuid = self._generate_uuid(doc['id'])
            
            # Создание эмбеддинга
            embedding = self.embedding_model.encode(doc['text']).tolist()
            
            # Подготовка payload
            payload = {
                'text': doc['text'],
                'user_id': doc['user_id'],
                'timestamp': doc['timestamp'],
                'original_id': doc['id']  # Сохраняем оригинальный ID
            }
            
            points_embeddings.append(PointStruct(
                id=doc_uuid,
                vector=embedding,
                payload=payload
            ))
            
            points_bm25.append(PointStruct(
                id=doc_uuid,
                vector=[1.0],
                payload=payload
            ))
            
            new_bm25_docs.append(self._tokenize_text(doc['text']))
            self.bm25_uuid_map[doc_uuid] = doc['id']
        
        # Добавление в Qdrant
        if points_embeddings:
            self.qdrant_embeddings.upsert(
                collection_name=QDRANT_COLLECTION_EMBEDDINGS,
                points=points_embeddings
            )
        
        if points_bm25:
            self.qdrant_bm25.upsert(
                collection_name=QDRANT_COLLECTION_BM25,
                points=points_bm25
            )
            
            # Обновление BM25
            if self.bm25 is None:
                self.bm25 = BM25Okapi(new_bm25_docs)
                self.bm25_documents = new_bm25_docs
                self.bm25_doc_ids = [doc['id'] for doc in documents]
            else:
                self.bm25_documents.extend(new_bm25_docs)
                self.bm25_doc_ids.extend([doc['id'] for doc in documents])
                self.bm25 = BM25Okapi(self.bm25_documents)
    
    def recalculate_bm25(self):
        """Полный пересчет BM25 по всем документам в базе"""
        self._load_bm25_data()
    
    def _build_filters(self, user_id: Optional[str] = None, 
                      start_timestamp: Optional[float] = None, 
                      end_timestamp: Optional[float] = None) -> Optional[Filter]:
        """Построение фильтров для Qdrant"""
        conditions = []
        
        if user_id:
            conditions.append(FieldCondition(
                key="user_id",
                match=MatchValue(value=user_id)
            ))
        
        if start_timestamp is not None or end_timestamp is not None:
            timestamp_range = {}
            if start_timestamp is not None:
                timestamp_range["gte"] = start_timestamp
            if end_timestamp is not None:
                timestamp_range["lte"] = end_timestamp
            
            conditions.append(FieldCondition(
                key="timestamp",
                range=Range(**timestamp_range)
            ))
        
        return Filter(must=conditions) if conditions else None
    
    def search(self, query: str, k: int = 10, m: int = 50,
               user_id: Optional[str] = None,
               start_timestamp: Optional[float] = None,
               end_timestamp: Optional[float] = None) -> List[Tuple[str, float]]:
        """
        Поиск с использованием RRF
        """
        filters = self._build_filters(user_id, start_timestamp, end_timestamp)
        
        # Поиск по эмбеддингам
        query_embedding = self.embedding_model.encode(query).tolist()
        embedding_results = self.qdrant_embeddings.search(
            collection_name=QDRANT_COLLECTION_EMBEDDINGS,
            query_vector=query_embedding,
            query_filter=filters,
            limit=m
        )
        
        # Конвертируем UUID обратно в оригинальные ID для эмбеддингов
        embedding_results_converted = []
        for result in embedding_results:
            original_id = result.payload.get('original_id', result.id)
            embedding_results_converted.append((original_id, result.score))
        
        # Поиск по BM25
        bm25_results = []
        if self.bm25:
            # Получаем все документы с фильтрацией
            all_points = self.qdrant_bm25.scroll(
                collection_name=QDRANT_COLLECTION_BM25,
                scroll_filter=filters,
                limit=10000
            )[0]
            
            filtered_docs = []
            filtered_original_ids = []
            
            for point in all_points:
                original_id = point.payload.get('original_id')
                if original_id and original_id in self.bm25_doc_ids:
                    idx = self.bm25_doc_ids.index(original_id)
                    filtered_docs.append(self.bm25_documents[idx])
                    filtered_original_ids.append(original_id)
            
            if filtered_docs:
                temp_bm25 = BM25Okapi(filtered_docs)
                tokenized_query = self._tokenize_text(query)
                bm25_scores = temp_bm25.get_scores(tokenized_query)
                
                # Сортируем по убыванию score
                bm25_indices = np.argsort(bm25_scores)[::-1][:m]
                bm25_results = [(filtered_original_ids[i], float(bm25_scores[i])) 
                               for i in bm25_indices if bm25_scores[i] > 0]
        
        # Объединение результатов с RRF
        return self._rrf_fusion(embedding_results_converted, bm25_results, k=k)
    
    def _rrf_fusion(self, embedding_results: List, bm25_results: List, k: int = 10, k_rrf: int = 60) -> List[Tuple[str, float]]:
        """
        Reciprocal Rank Fusion
        """
        ranked_lists = []
        
        # Ранжирование для эмбеддингов
        if embedding_results:
            embedding_rank = {doc_id: rank for rank, (doc_id, _) in enumerate(embedding_results)}
            ranked_lists.append(embedding_rank)
        
        # Ранжирование для BM25
        if bm25_results:
            bm25_rank = {doc_id: rank for rank, (doc_id, _) in enumerate(bm25_results)}
            ranked_lists.append(bm25_rank)
        
        # RRF подсчет
        rrf_scores = {}
        for ranking in ranked_lists:
            for doc_id, rank in ranking.items():
                if doc_id not in rrf_scores:
                    rrf_scores[doc_id] = 0
                rrf_scores[doc_id] += 1.0 / (k_rrf + rank + 1)
        
        # Сортировка по RRF score
        sorted_results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)[:k]
        return sorted_results
    
    def delete_documents(self, user_id: Optional[str] = None,
                        start_timestamp: Optional[float] = None,
                        end_timestamp: Optional[float] = None,
                        doc_ids: Optional[List[str]] = None):
        """
        Удаление документов по фильтрам
        """
        filters = None
        if user_id or start_timestamp is not None or end_timestamp is not None:
            filters = self._build_filters(user_id, start_timestamp, end_timestamp)
        
        # Если переданы конкретные ID, конвертируем в UUID
        uuid_ids = None
        if doc_ids:
            uuid_ids = [self._generate_uuid(doc_id) for doc_id in doc_ids]
        
        # Удаление из обеих баз
        self.qdrant_embeddings.delete(
            collection_name=QDRANT_COLLECTION_EMBEDDINGS,
            points_selector=uuid_ids if uuid_ids else filters
        )
        
        self.qdrant_bm25.delete(
            collection_name=QDRANT_COLLECTION_BM25,
            points_selector=uuid_ids if uuid_ids else filters
        )
        
        # Пересчет BM25 после удаления
        self.recalculate_bm25()
    
    def get_document_texts(self, doc_ids: List[str]) -> List[Tuple[str, str]]:
        """
        Получение текстов документов по их ID
        """
        results = []
        # Конвертируем оригинальные ID в UUID для поиска
        uuid_ids = [self._generate_uuid(doc_id) for doc_id in doc_ids]
        
        points = self.qdrant_bm25.retrieve(
            collection_name=QDRANT_COLLECTION_BM25,
            ids=uuid_ids
        )
        
        for point in points:
            if point and point.payload:
                original_id = point.payload.get('original_id', point.id)
                results.append((original_id, point.payload['text']))
        return results

def run_mistral(messages, user_format=True, model="mistral-medium-latest", api_key=None):
    """Функция для запросов к Mistral API"""
    client = Mistral(api_key=api_key)
    if user_format:
        messages = [
            {"role": "user", "content": messages}
        ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return chat_response.choices[0].message.content

def user_message(inquiry, messages=[]):
    """
    Форматирование промта для LLM
    """
    formatted_messages = "\n".join([f"<{{{msg[0]}, {msg[1]}}}>" for msg in messages]) if messages else "Нет сообщений."

    user_message = f"""
РОЛЬ: Ты помощник по поиску информации в чатах Telegram.
ЗАДАЧА: На основе запроса пользователя и предоставленных сообщений составь ответ.
ФОРМАТ ДАННЫХ:
История сообщений в формате:
<{{message_id, message_text}}>
Каждое сообщение на отдельной строке.

ПРАВИЛА:
- Используй только предоставленные сообщения для ответа.
- Если нет подходящей информации, ответь "Извините, я не смог найти информацию по вашему запросу."
- Будь краток и точен.
- Не обязательно использовать все предоставленные сообщения, выбери только релевантные.

ФОРМАТ ОТВЕТА:
Ответ: [текст ответа]
Использованные сообщения: <{{message_id1, message_id2, ...}}>

История сообщений:
{formatted_messages}

Запрос пользователя: {inquiry}
"""
    return user_message


ТЕСТЫ.

каспом выделил на что обратить внимание 

In [43]:
# Пример использования
def main():
    # Инициализация системы
    #ПОМИОТРЕТЬ КАК ИНИЦИАЛИЗИРУЕТСЯ БДШКA (чтобы каждый раз не пересоздавать)
    rag_system = TelegramRAGSystem(data_dir="./rag_data", mistral_api_key="your_mistral_api_key")
    
    # Добавление тестовых документов
    test_documents = [
        {
            'id': 'msg_001',
            'text': 'Встречаемся завтра в 15:00 у главного входа',
            'user_id': 'user1',
            'timestamp': 1672531200
        },
        {
            'id': 'msg_002', 
            'text': 'Не забудьте взять документы на собрание',
            'user_id': 'user2',
            'timestamp': 1672617600
        },
        {
            'id': 'msg_003',
            'text': 'Отмена, Встреча перенесена с 15:00 на следующий понедельник. тогда и заключим сделку',
            'user_id': 'user1',
            'timestamp': 167200000
        },
        {
            'id': 'msg_004',
            'text': 'Какие документы нужны для встречи?',
            'user_id': 'user3',
            'timestamp': 1672790400
        },
        {
            'id': 'msg_005',
            'text': 'Встречаемся вечером у клуба',
            'user_id': 'user2',
            'timestamp': 1673000000
        },
        {
            'id': 'msg_006',
            'text': 'Идем гулять сегодня в парк',
            'user_id': 'user1',
            'timestamp': 1672963200
        },


    ]
    rag_system.add_documents(test_documents)
    
    # ПОСМОТРЕТЬ ФОРМАТЬ ВРЕМЕНИ, ПОДдЕРЖАТЬ фИЛТРАЦию ПО ВРЕМЕНИ, УЗЕРУ (функция предусматривает)
    results = rag_system.search("когда встреча?", k=5, end_timestamp=1673000000-1)
    print("Результаты поиска:", results)
    
    # ВОЗМОЖНО СТОИТ ЗАВЕСТИ ОТДЕЛЬНУЮ БДШКУ ПОД ТЕКСТА
    doc_ids = [doc_id for doc_id, score in results]
    messages = rag_system.get_document_texts(doc_ids)
    print("Найденные сообщения:", messages)
    
    
    prompt = user_message("во сколько идем тусить?", messages)
    API_KEY = 'your_mistral_api_key'
    #ПОДДЕРЖАТЬ РАЗНЫЕ МОДЕЛИ 
    response = run_mistral(prompt, api_key=API_KEY, model="mistral-tiny")
    print("Ответ LLM:", response)

if __name__ == "__main__":
    main()

/var/folders/qn/zdfk35s17rg427bmt4dyz37r0000gn/T/ipykernel_88339/2858080996.py:194: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  embedding_results = self.qdrant_embeddings.search(


Результаты поиска: [('msg_004', 0.03278688524590164), ('msg_003', 0.0315136476426799), ('msg_006', 0.016129032258064516), ('msg_001', 0.015873015873015872), ('msg_002', 0.015625)]
Найденные сообщения: [('msg_004', 'Какие документы нужны для встречи?'), ('msg_003', 'Отмена, Встреча перенесена с 15:00 на следующий понедельник. тогда и заключим сделку'), ('msg_006', 'Идем гулять сегодня в парк'), ('msg_001', 'Встречаемся завтра в 15:00 у главного входа'), ('msg_002', 'Не забудьте взять документы на собрание')]
Ответ LLM: Ответ: Сегодня мы идем гулять в 15:00.
Использованные сообщения: <{msg_004, Какие документы нужны для встречи?}>
<{msg_006, Идем гулять сегодня в парк}>


Можно проверить подключение к мистралю. выводятся все доступные модели.


In [ ]:
# Access the secret
api_key = API_KEY

if not api_key:
    raise ValueError("❌ MISTRAL_API_KEY not found in Colab secrets!")
else:
    print("✅ API key loaded successfully from Colab secrets!")

# Initialize Mistral client
client = Mistral(api_key=api_key)

# Test connection
def test_connection():
    try:
        models = client.models.list()
        print("✅ Connected successfully!")
        print(f"Available models: {[m.id for m in models.data]}")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        print("💡 If key is not active yet, wait a few minutes and try again")

test_connection()